In [1]:
import pandas as pd
import numpy as np

# load data
DATA_FILE = 'data/train.csv'
TEST_FILE = 'data/test.csv'
df = pd.read_csv(DATA_FILE)
TRAIN_DATA_CNT = len(df)
test_df = pd.read_csv('data/test.csv')

print df.head()
print df.shape

y_data_df = df['SalePrice']
print y_data_df.shape
x_data_df = df.drop('SalePrice',axis=1)
all_df = x_data_df.copy()
all_data_df = all_df.append(test_df) # combine train and test for encode labeler

cut_feat_cols = []
for col in all_data_df.columns:
    null_cnt = all_data_df[col].isnull().sum()
    print null_cnt,col
    if null_cnt>10:
        cut_feat_cols.append(col)
all_data_df = all_data_df.drop(cut_feat_cols,axis=1)

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities    ...     PoolArea PoolQC Fence MiscFeature MiscVal  \
0         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
1         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
2         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
3         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   
4         Lvl    AllPub    ...            0    NaN   NaN         NaN       0   

  MoSold YrSold  SaleType  SaleCondition  SalePrice  
0      2   2008     

In [2]:
all_data_df = all_data_df.apply(lambda x:x.fillna(x.value_counts().index[0])) # fillna with most frequent item

print all_data_df.shape
print all_data_df.head()

tmp_list = []
for v,c in zip(all_data_df.values[0],all_data_df.columns):
    try:
        float(v)
    except:
        print v
        tmp_list.append(c)
print tmp_list

(2919, 62)
   Id  MSSubClass MSZoning  LotArea Street LotShape LandContour Utilities  \
0   1          60       RL     8450   Pave      Reg         Lvl    AllPub   
1   2          20       RL     9600   Pave      Reg         Lvl    AllPub   
2   3          60       RL    11250   Pave      IR1         Lvl    AllPub   
3   4          70       RL     9550   Pave      IR1         Lvl    AllPub   
4   5          60       RL    14260   Pave      IR1         Lvl    AllPub   

  LotConfig LandSlope      ...      OpenPorchSF EnclosedPorch 3SsnPorch  \
0    Inside       Gtl      ...               61             0         0   
1       FR2       Gtl      ...                0             0         0   
2    Inside       Gtl      ...               42             0         0   
3    Corner       Gtl      ...               35           272         0   
4       FR2       Gtl      ...               84             0         0   

  ScreenPorch PoolArea  MiscVal  MoSold  YrSold  SaleType SaleCondition  
0

In [3]:
# data preprocessing
# should be done by read description 
cls_cols = tmp_list 
print cls_cols
print len(cls_cols)
x_col_df = all_data_df[cls_cols] 
x_num_df = all_data_df.drop(cls_cols,axis=1)
print 'col df',x_col_df.shape
print 'num df',x_num_df.shape

# label encode
from sklearn import preprocessing
label_trans_list = []
x_col_data_list = []
for col in cls_cols:
    label_encoder = preprocessing.LabelEncoder()
    col_data = label_encoder.fit_transform(x_col_df[col])
    label_trans_list.append(label_encoder)
    x_col_data_list.append(col_data)


x_col_data_new = np.array(x_col_data_list).transpose()
final_all_data = np.hstack((x_num_df.values,x_col_data_new))
print final_all_data.shape
print '-'*20
print 'trans',final_all_data[0]
print 'org',all_data_df.values[0]
print '-'*20
final_x_data = final_all_data[:TRAIN_DATA_CNT]
final_x_test_data = final_all_data[TRAIN_DATA_CNT:]
print 'train x data',final_x_data.shape
print 'test x data',final_x_test_data.shape
final_y_data = y_data_df.values
print 'train y data',final_y_data.shape

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']
28
col df (2919, 28)
num df (2919, 34)
(2919, 62)
--------------------
trans [  1.00000000e+00   6.00000000e+01   8.45000000e+03   7.00000000e+00
   5.00000000e+00   2.00300000e+03   2.00300000e+03   7.06000000e+02
   0.00000000e+00   1.50000000e+02   8.56000000e+02   8.56000000e+02
   8.54000000e+02   0.00000000e+00   1.71000000e+03   1.00000000e+00
   0.00000000e+00   2.00000000e+00   1.00000000e+00   3.00000000e+00
   1.00000000e+00   8.00000000e+00   0.00000000e+00   2.00000000e+00
   5.48000000e+02   0.00000000e+00   6.10000000e+01   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.000000

In [4]:
# try random forest model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn import metrics
import time
X_train, X_test, y_train, y_test = train_test_split(final_x_data, final_y_data, test_size=0.5, random_state=2333)
print np.mean(final_y_data)
print np.var(final_y_data)

import math

#A function to calculate Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y, y_pred):
    y = np.clip(y,0,max(y))
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

def test_model(model):
    print '-'*20
    model.fit(X_train,y_train)
    # test
    res = model.predict(X_test)
    print 'mae', metrics.mean_absolute_error(res, y_test),'rmsle',rmsle(res,y_test),

    # test on all
    s = time.time()
    res = model.predict(final_x_data)
    print '{} data time cost {}'.format(len(final_x_data), time.time() - s),
    print 'mae', metrics.mean_absolute_error(res, final_y_data),'rmsle',rmsle(res,y_test)




model = RandomForestRegressor()
test_model(model)

for m in [LinearRegression(),Lasso(),Ridge()]:
    test_model(m)

180921.19589
6306788585.35
--------------------
mae 20775.1058904 rmsle 0.167827210455 1460 data time cost 0.0253591537476 mae 14207.2033562 rmsle 0.387624082328
--------------------
mae 21022.2584388 rmsle 0.182503098768 1460 data time cost 0.000485181808472 mae 18984.5183884 rmsle 0.423182987936
--------------------
mae 21021.4972488 rmsle 0.182486922935 1460 data time cost 0.000482082366943 mae 18984.3391561 rmsle 0.423191113145
--------------------
mae 21058.9160539 rmsle 0.183905340275 1460 data time cost 0.000536918640137 mae 19010.736536 rmsle 0.423618392113


In [ ]:
print 'check xgboost'

from xgboost import XGBRegressor
model = XGBRegressor()
test_model(model)

model = XGBRegressor(n_estimators=90)
test_model(model)

model = XGBRegressor(n_estimators=90,max_depth=2)
test_model(model)

model = XGBRegressor(n_estimators=90,max_depth=4)
test_model(model)

model = XGBRegressor(n_estimators=90,max_depth=8)
test_model(model)

model = XGBRegressor(n_estimators=90,subsample=0.7)
test_model(model)

model = XGBRegressor(n_estimators=90,subsample=0.8)
test_model(model)

model = XGBRegressor(n_estimators=90,subsample=0.9)
test_model(model)

model = XGBRegressor(n_estimators=90,learning_rate=0.15)
test_model(model)

model = XGBRegressor(n_estimators=90,learning_rate=0.4)
test_model(model)

check xgboost
--------------------


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


mae 18185.2211526 rmsle 0.147010198608 1460 data time cost 0.0122878551483 mae 13575.0131582 rmsle 0.398861818099
--------------------
mae 18312.8869702 rmsle 0.148429094824 1460 data time cost 0.00569200515747 mae 13812.8198363 rmsle 0.397900867389
--------------------
mae 18872.7196597 rmsle 0.157090388453 1460 data time cost 0.00426602363586 mae 15610.0926423 rmsle 0.393436792299
--------------------
mae 18093.1380351 rmsle 0.147874394966 1460 data time cost 0.00761795043945 mae 12335.3213854 rmsle 0.399382924729
--------------------
mae 18559.343857 rmsle 0.151111652424 1460 data time cost 0.0291259288788 mae 9995.20440122 rmsle 0.399243311905
--------------------
mae 17942.3532053 rmsle 0.148184503729 1460 data time cost 0.00524497032166 mae 13515.3389929 rmsle 0.398261499255
--------------------
mae 18240.4128639 rmsle 0.149417948127 1460 data time cost 0.00850820541382 mae 13790.9949727 rmsle 0.398116603842
--------------------
mae 18063.4551637 rmsle 0.147952900711 1460 data ti

In [ ]:
# try grid search
from sklearn.grid_search import GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(final_x_data, final_y_data, test_size=0.5, random_state=2333)
params = {'n_estimators':list(range(20,160,20)),
          'max_depth':[2,3,4,5,6],
          'subsample':[0.4,0.6,0.8],
          'min_child_weight':[1,1.2,1.4],
          'learning_rate':[0.02,0.04,0.06,1]
         }
xgb = XGBRegressor()
reg = GridSearchCV(estimator=xgb,param_grid=params)
reg.fit(final_x_data,final_y_data) # grid test on all data
print reg.best_estimator_
def test_model_on_data(model):
    print '-'*20
    # model.fit(X_train,y_train)
    print 'new_test'
    # test
    res = model.predict(X_test)
    print 'mae', metrics.mean_absolute_error(res, y_test),'rmsle',rmsle(res,y_test)

    # test on all
    s = time.time()
    res = model.predict(final_x_data)
    print '{} data time cost {}'.format(len(final_x_data), time.time() - s)
    print 'mae', metrics.mean_absolute_error(res, final_y_data),'rmsle',rmsle(res,y_test)
test_model_on_data(reg)

/usr/local/lib/python2.7/dist-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:
# run on test data and save to csv
final_model = XGBRegressor(n_estimators=90,max_depth=4,subsample=0.8,learning_rate=0.06)
final_model.fit(X_train,y_train)
final_y = final_model.predict(final_x_test_data)
print final_y
res_df = test_df[['Id']]
res_df['SalePrice'] = final_y
print res_df.head()
res_df.to_csv('my_res_cut_feature_part_train.csv',index=False)

final_model = XGBRegressor(n_estimators=90,max_depth=4,subsample=0.8,learning_rate=0.06)
final_model.fit(final_x_data,final_y_data)
final_y = final_model.predict(final_x_test_data)
print final_y
res_df = test_df[['Id']]
res_df['SalePrice'] = final_y
print res_df.head()
res_df.to_csv('my_res_cut_feature_all_train.csv',index=False)
